# HW 3: Neural Machine Translation

In this homework you will build a full neural machine translation system using an attention-based encoder-decoder network to translate from German to English. The encoder-decoder network with attention forms the backbone of many current text generation systems. See [Neural Machine Translation and Sequence-to-sequence Models: A Tutorial](https://arxiv.org/pdf/1703.01619.pdf) for an excellent tutorial that also contains many modern advances.

## Goals


1. Build a non-attentional baseline model (pure seq2seq as in [ref](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf)). 
2. Incorporate attention into the baseline model ([ref](https://arxiv.org/abs/1409.0473) but with dot-product attention as in class notes).
3. Implement beam search: review/tutorial [here](http://www.phontron.com/slides/nlp-programming-en-13-search.pdf)
4. Visualize the attention distribution for a few examples. 

Consult the papers provided for hyperparameters, and the course notes for formal definitions.

This will be the most time-consuming assignment in terms of difficulty/training time, so we recommend that you get started early!

## Setup

This notebook provides a working definition of the setup of the problem itself. Feel free to construct your models inline, or use an external setup (preferred) to build your system.

In [1]:
# Text text processing library and methods for pretrained word embeddings
from torchtext import data
from torchtext import datasets
import torch as t

We first need to process the raw data using a tokenizer. We are going to be using spacy, which can be installed via:  
  `[sudo] pip install spacy`  
  
Tokenizers for English/German can be installed via:  
  `[sudo] python -m spacy download en`  
  `[sudo] python -m spacy download de`
  
This isn't *strictly* necessary, and you can use your own tokenization rules if you prefer (e.g. a simple `split()` in addition to some rules to acccount for punctuation), but we recommend sticking to the above.

In [2]:
import spacy
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]


Note that we need to add the beginning-of-sentence token `<s>` and the end-of-sentence token `</s>` to the 
target so we know when to begin/end translating. We do not need to do this on the source side.

In [3]:
BOS_WORD = '<s>'
EOS_WORD = '</s>'
DE = data.Field(tokenize=tokenize_de)
EN = data.Field(tokenize=tokenize_en, init_token = BOS_WORD, eos_token = EOS_WORD) # only target needs BOS/EOS

Let's download the data. This may take a few minutes.

**While this dataset of 200K sentence pairs is relatively small compared to others, it will still take some time to train. So we are going to be only working with sentences of length at most 20 for this homework. Please train only on this reduced dataset for this homework.**

In [14]:
MAX_LEN = 20
train, val, test = datasets.IWSLT.splits(exts=('.de', '.en'), fields=(DE, EN), 
                                         filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and 
                                         len(vars(x)['trg']) <= MAX_LEN)
print(train.fields)
print(len(train))
print(vars(train[0]))

downloading de-en.tgz
.data\iwslt\de-en\IWSLT16.TED.dev2010.de-en.de.xml
.data\iwslt\de-en\IWSLT16.TED.dev2010.de-en.en.xml
.data\iwslt\de-en\IWSLT16.TED.tst2010.de-en.de.xml
.data\iwslt\de-en\IWSLT16.TED.tst2010.de-en.en.xml
.data\iwslt\de-en\IWSLT16.TED.tst2011.de-en.de.xml
.data\iwslt\de-en\IWSLT16.TED.tst2011.de-en.en.xml
.data\iwslt\de-en\IWSLT16.TED.tst2012.de-en.de.xml


UnicodeEncodeError: 'charmap' codec can't encode character '\u200c' in position 183: character maps to <undefined>

Now we build the vocabulary and convert the text corpus into indices. We are going to be replacing tokens that occurred less than 5 times with `<unk>` tokens, and take the rest as our vocab.

In [ ]:
MIN_FREQ = 5
DE.build_vocab(train.src, min_freq=MIN_FREQ)
EN.build_vocab(train.trg, min_freq=MIN_FREQ)
print(DE.vocab.freqs.most_common(10))
print("Size of German vocab", len(DE.vocab))
print(EN.vocab.freqs.most_common(10))
print("Size of English vocab", len(EN.vocab))
print(EN.vocab.stoi["<s>"], EN.vocab.stoi["</s>"]) #vocab index for <s>, </s>

Now we split our data into batches as usual. Batching for MT is slightly tricky because source/target will be of different lengths. Fortunately, `torchtext` lets you do this by allowing you to pass in a `sort_key` function. This will minimizing the amount of padding on the source side, but since there is still some padding you will inadvertendly "attend" to these padding tokens. 

One way to get rid of padding is to pass a binary `mask` vector to your attention module so its attention score (before the softmax) is minus infinity for the padding token. Another way (which is how we do it for our projects, e.g. opennmt) is to manually sort data into batches so that each batch has exactly the same source length (this means that some batches will be less than the desired batch size, though).

However, for this homework padding won't matter too much, so it's fine to ignore it.

In [ ]:
BATCH_SIZE = 32
train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=BATCH_SIZE, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src))

Let's check to see that the BOS/EOS token is indeed appended to the target (English) sentence.

In [ ]:
batch = next(iter(train_iter))
print("Source")
print(batch.src)
print("Target")
print(batch.trg)


Success! Now that we've processed the data, we are ready to begin modeling.

## Assignment

Now it is your turn to build the models described at the top of the assignment. 

When a model is trained, use the following test function to produce predictions, and then upload to the kaggle competition: https://www.kaggle.com/c/cs287-hw3-s18/

For the final Kaggle test, we will provide the source sentence, and you are to predict the **first three words of the target sentence**. The source sentence can be found under `source_test.txt`

In [1]:
!head source_test.txt

Als ich in meinen 20ern war , hatte ich meine erste Psychotherapie-Patientin .
Ich war Doktorandin und studierte Klinische Psychologie in Berkeley .
Sie war eine 26-jährige Frau namens Alex .
Und als ich das hörte , war ich erleichtert .
Meine Kommilitonin bekam nämlich einen Brandstifter als ersten Patienten .
Und ich bekam eine Frau in den 20ern , die über Jungs reden wollte .
Das kriege ich hin , dachte ich mir .
Aber ich habe es nicht hingekriegt .
Arbeit kam später , Heiraten kam später , Kinder kamen später , selbst der Tod kam später .
Leute in den 20ern wie Alex und ich hatten nichts als Zeit .


Similar to HW1, you are to predict the 100 most probable 3-gram that will begin the target sentence. The submission format will be as follows, where each word in the 3-gram will be separated by "|", and each 3-gram will be separated by space. For example, here is what an example submission might look like with 5 most-likely 3-grams (instead of 100).

```
id,word
1,Newspapers|talk|about When|I|was Researchers|call|the Twentysomethings|like|Alex But|before|long
2,That|'s|what Newspapers|talk|about You|have|robbed It|'s|realizing My|parents|wanted
3,We|forget|how We|think|about Proust|actually|links Does|any|other This|is|something
4,But|what|do And|it|'s They|'re|on My|name|is It|only|happens
```

When you print out your data, you will need to escape quotes and commas with the following command so that Kaggle does not complain. 

In [3]:
def escape(l):
    return l.replace("\"", "<quote>").replace(",", "<comma>")

You should perform your hyperparameter search/early stopping/write-up based on perplexity, not the above metric. (In practice, people use a metric called [BLEU](https://www.aclweb.org/anthology/P02-1040.pdf), which is roughly a geometric average of 1-gram, 2-gram, 3-gram, 4-gram precision, with a brevity penalty for producing translations that are too short.)

Finally, as always please put up a (short) write-up following the template provided in the repository:  https://github.com/harvard-ml-courses/cs287-s18/blob/master/template/


In [10]:
import torch as t
from utils import variable
import numpy as np
import torch.nn.functional as F

x = variable(np.random.normal(size=(10,32,40)))
F.softmax(x.permute(1,0,2)).permute(1,0,2)

Variable containing:
(0 ,.,.) = 
  0.0058  0.0272  0.0966  ...   0.0186  0.0085  0.0117
  0.0681  0.0377  0.0280  ...   0.0225  0.0105  0.0154
  0.0436  0.0264  0.0042  ...   0.0058  0.0443  0.0749
           ...             ⋱             ...          
  0.0243  0.0972  0.0200  ...   0.0044  0.0442  0.0132
  0.0132  0.0300  0.0315  ...   0.0035  0.0130  0.0102
  0.0347  0.0062  0.0148  ...   0.0113  0.0100  0.0065

(1 ,.,.) = 
  0.0259  0.0691  0.0082  ...   0.0556  0.0084  0.0058
  0.0122  0.0067  0.0034  ...   0.0632  0.0026  0.0224
  0.0240  0.0056  0.0252  ...   0.2280  0.0153  0.0090
           ...             ⋱             ...          
  0.0172  0.0487  0.0425  ...   0.0023  0.0242  0.0240
  0.1516  0.0452  0.0177  ...   0.0114  0.0038  0.0319
  0.0256  0.0048  0.0497  ...   0.0084  0.0885  0.0480

(2 ,.,.) = 
  0.0158  0.0112  0.0227  ...   0.0515  0.0475  0.0091
  0.2038  0.0085  0.0301  ...   0.0094  0.0191  0.0578
  0.0378  0.0382  0.0079  ...   0.0231  0.0209  0.0042
      

In [43]:
lstm = t.nn.LSTM(50, 100, dropout=.2, num_layers=1, bidirectional=True, batch_first=True)
from utils import variable
x = variable(t.zeros(10, 32, 50))
out, h = lstm(x)

print(out.size())
print(h[0].size(), h[1].size())


torch.Size([10, 32, 200])
torch.Size([2, 10, 100]) torch.Size([2, 10, 100])


In [22]:
h[0].view(10,200)

Variable containing:
 0.0117 -0.0371  0.0068  ...   0.0045 -0.0319  0.0171
 0.0117 -0.0371  0.0068  ...   0.0045 -0.0319  0.0171
 0.0117 -0.0371  0.0068  ...   0.0045 -0.0319  0.0171
          ...             ⋱             ...          
-0.0376  0.0367  0.0461  ...   0.0180  0.0400 -0.0091
-0.0376  0.0367  0.0461  ...   0.0180  0.0400 -0.0091
-0.0376  0.0367  0.0461  ...   0.0180  0.0400 -0.0091
[torch.FloatTensor of size 10x200]

In [39]:
t.sum((out[-1] == t.cat([h[0][1], h[0][0]], 0)).long())

C:\ProgramData\Anaconda3\lib\site-packages\torch\autograd\_functions\compare.py:17: UserWarning: self and other not broadcastable, but have the same number of elements.  Falling back to deprecated pointwise behavior.
  mask = getattr(a, cls.fn_name)(b)


Variable containing:
 500
[torch.LongTensor of size 1]

In [45]:
import numpy as np
t.baddbmm(variable(np.ones((10, 20, 100))) + variable(2*np.ones((10, 100)))

RuntimeError: inconsistent tensor size, expected r_ [10 x 20 x 100], t [10 x 20 x 100] and src [10 x 100] to have the same number of elements, but got 20000, 20000 and 1000 elements respectively at d:\projects\pytorch\torch\lib\th\generic/THTensorMath.c:887